In [124]:
from mido import MidiFile
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

from music21 import converter, instrument, note, chord, stream
import pypianoroll

In [116]:
root_dir = '/home/fsantiago/research/music-data'
data_dir = root_dir + '/Lakh Piano Dataset/lpd_5/lpd_5_cleansed'
midi_dir = '/home/fsantiago/research/music-data/Lakh Piano Dataset/lpd_5_midi/'
lakh_dir = '/home/fsantiago/research/music-data/Lakh Piano Dataset'


In [108]:
cleansed_ids = pd.read_csv(os.path.join(root_dir, 'Lakh Piano Dataset', 'cleansed_ids.txt'), delimiter = '    ', header = None)
lpd_to_msd_ids = {a:b for a, b in zip(cleansed_ids[0], cleansed_ids[1])}
msd_to_lpd_ids = {a:b for a, b in zip(cleansed_ids[1], cleansed_ids[0])}

/tmp/ipykernel_22115/1716945500.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  cleansed_ids = pd.read_csv(os.path.join(root_dir, 'Lakh Piano Dataset', 'cleansed_ids.txt'), delimiter = '    ', header = None)


In [109]:
RESULTS_PATH = os.path.join(root_dir, 'Lakh Piano Dataset', 'Metadata')


# Utility functions for retrieving paths
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)


def msd_id_to_h5(msd_id):
    """Given an MSD ID, return the path to the corresponding h5"""
    return os.path.join(RESULTS_PATH, 'lmd_matched_h5',
                        msd_id_to_dirs(msd_id) + '.h5')

# Load the midi npz file from the LMD cleansed folder
def get_midi_npz_path(msd_id, midi_md5):
    return os.path.join(data_dir,
                        msd_id_to_dirs(msd_id), midi_md5 + '.npz')
    
# # Load the midi file from the Music Dataset folder
# def get_midi_path(msd_id, midi_md5):
#     return os.path.join(music_dataset_lpd_dir,
#                         msd_id_to_dirs(msd_id), midi_md5 + '.mid')

In [110]:
for msd_file_name in list(lpd_to_msd_ids.values()):
    lpd_file_name = msd_to_lpd_ids[msd_file_name]

    npz_path = get_midi_npz_path(msd_file_name, lpd_file_name)

    pianoroll = pypianoroll.load(npz_path)
    midi = pypianoroll.to_pretty_midi(pianoroll)

    pypianoroll.write(f"{midi_dir}/{lpd_file_name}.mid", pianoroll)

In [120]:
for _, _, t in os.walk(f"{midi_dir}"):
    print(t)

['1c6f05caec9a219febc7065e4716cb79.mid', '9a0208c5eefa89ef1d05a8a2a06acdc8.mid', 'daca4a012cc3347128cea35230ec5cd1.mid', 'c73cf18220f2eaf5dea6eb921118a2e1.mid', '2f97a8df16535b2e05ab6192563e1561.mid', '9cd28edbc06965df3cba5bf0e87c470f.mid', 'f9b22373e95e52fd734aff6abe2b64f4.mid', 'd9fa607a262edebfd57758922dda23bd.mid', 'eb12878d4120337bd90fd460f88aa530.mid', 'a244f8e4328af6b2718b804156556885.mid', '4cf1ac8a61ed127ffa4e9d77ee87441b.mid', '497f24c6f3975a79cac53fad789e5bf5.mid', '27d810d22f8ad40968919d3ed2949eca.mid', '2225b51ee440f6165e47e420a836c2f2.mid', 'eef66396b48798dc4fa0be8d05b81d56.mid', '89227f90ffcccba4af80a1f02fddaf6a.mid', 'c232e7bc853c5d23c7747431cde0af84.mid', '7d1da8b452b198623e67c19c4d85413c.mid', '9b887a42348ff30599a40a449b12630f.mid', '5d01b21c9c3987405a5ee24f3916cca9.mid', '5e3e8311aa2e9e19a416576ea98fa5c7.mid', 'f30af7b4c3bf67a1a2b3dc09c1680447.mid', 'ff8163483bfde759f8af669b122a22ed.mid', 'da26449830718e812406ac3fdb756235.mid', '6991d81202e4c6973d6fb603246819bd.mid',

In [131]:
i = 0
notes = set()

for _, _, files in os.walk(f"{midi_dir}"):
    for file in tqdm(files[:1000]):
        midi = converter.parse(f"{midi_dir}/{file}")

        s2 = instrument.partitionByInstrument(midi)
        piano_part = None
        # Filter for  only the piano part
        instr = instrument.Piano
        for part in s2:
            if isinstance(part.getInstrument(), instr):
                piano_part = part

        notes_song = []
        if piano_part: # Some songs somehow have no piano parts
            for element in piano_part:
                if isinstance(element, note.Note):
                    # Return the pitch of the single note
                    notes.add(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    # Returns the normal order of a Chord represented in a list of integers
                    notes.add('.'.join(str(n) for n in element.normalOrder))
        i+=1

  0%|          | 0/1000 [00:00<?, ?it/s]


TypeError: unhashable type: 'Note'

In [ ]:
notes = []

i = 0
for msd_file_name in list(lpd_to_msd_ids.values())[:50]:
  lpd_file_name = msd_to_lpd_ids[msd_file_name]

  # Get the NPZ path
  npz_path = get_midi_npz_path(msd_file_name, lpd_file_name)

  multitrack = pypianoroll.load(npz_path)
  pm = pypianoroll.to_pretty_midi(multitrack)
  new_midi_path = npz_path[:-4] + '.mid'
  pypianoroll.write(new_midi_path, multitrack)
  # Get the MIDI path (should already be generated)
  new_midi_path = npz_path[:-4] + '.mid'
  midi = converter.parse(new_midi_path)

  s2 = instrument.partitionByInstrument(midi)
  piano_part = None
  # Filter for  only the piano part
  instr = instrument.Piano
  for part in s2:
    if isinstance(part.getInstrument(), instr):
      piano_part = part

  notes_song = []
  if piano_part: # Some songs somehow have no piano parts
    for element in piano_part:
      if isinstance(element, note.Note):
        # Return the pitch of the single note
          notes_song.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
        # Returns the normal order of a Chord represented in a list of integers
          notes_song.append('.'.join(str(n) for n in element.normalOrder))

  notes.append(notes_song)
  i+=1
  print(i)

In [91]:
test = list(filter(lambda x: len(x) > 0, notes))

In [94]:
lpd_file_name = msd_to_lpd_ids[msd_file_name]
get_midi_npz_path(msd_file_name, lpd_file_name)

'/home/fsantiago/research/music-data/Lakh Piano Dataset/lpd_5/lpd_5_cleansed/X/W/D/TRXWDFM12903CA4461/8374638a3e8d7bbea4ff0616ef67942e.npz'